# 📦 Instalación de Librerías Necesarias para el Proyecto

Este proyecto requiere varias bibliotecas para el análisis y procesamiento de datos e imágenes. A continuación, se listan y describen:

- **pandas**: Manipulación y análisis de datos estructurados (tablas tipo DataFrame).
- **numpy**: Operaciones matemáticas avanzadas y manejo de arreglos multidimensionales.
- **matplotlib**: Visualización básica de gráficos e imágenes.
- **seaborn**: Visualización estadística avanzada basada en matplotlib.
- **opencv-python (cv2)**: Procesamiento de imágenes y visión por computadora.
- **scikit-image**: Lectura, filtrado y análisis de imágenes.
- **plotly**: Visualización interactiva y dinámica de gráficos.
- **kaggle**: Interfaz para descargar datasets desde [Kaggle.com](https://www.kaggle.com).

### ⏱ Tiempo estimado de instalación:
- Conexión buena: 3 a 7 minutos.
- Conexión lenta o Colab recién iniciado: 10 a 15 minutos.

### 💡 Recomendación:
- Si no va a trabajar directamente con imágenes, puede comentar o eliminar las siguientes librerías:
  - `opencv-python`
  - `scikit-image`

### ⚠️ Importante:
- La librería `kaggle` es necesaria solo si va a descargar datasets directamente desde Kaggle.
- Asegúrese de haber subido el archivo `kaggle.json` al entorno antes de autenticar con `KaggleApi`.

---


In [ ]:

!pip install -q kaggle
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install opencv-python
!pip install scikit-image
!pip install plotly


### 📥 Importación de librerías necesarias

Este bloque importa todas las librerías esenciales que se usarán a lo largo del proyecto. **Puede tardar alrededor de un minuto** en completarse, dependiendo del entorno y los recursos de tu máquina, ya que algunas son pesadas.

A continuación se explica el propósito de cada librería:

- **`os`**  
  Permite interactuar con el sistema operativo. Aquí se usa para cambiar la ubicación en la que `kaggle.json` será buscado (en este caso, la raíz del proyecto).

- **`kaggle.api.kaggle_api_extended.KaggleApi`**  
  Se utiliza para autenticar y descargar datasets directamente desde [Kaggle](https://www.kaggle.com).

- **`pandas` (`pd`)**  
  Manejo y análisis de datos en estructuras tipo tabla (`DataFrame`).

- **`numpy` (`np`)**  
  Cálculo numérico y manejo eficiente de arreglos multidimensionales.

- **`shutil`**  
  Permite copiar, mover o eliminar archivos y carpetas; útil para organizar las imágenes.

- **`cv2` (OpenCV)**  
  Librería de visión por computadora para leer, modificar y procesar imágenes.

- **`matplotlib.pyplot` (`plt`)**  
  Visualización de datos mediante gráficos estáticos y personalizables.

- **`seaborn` (`sns`)**  
  Complemento de `matplotlib` para gráficos estadísticos más atractivos y fáciles de generar.

- **`plotly.graph_objs` (`go`)**  
  Generación de gráficos interactivos (por ejemplo, líneas, barras, mapas de calor).

- **`skimage.io`**  
  Lectura y escritura de imágenes desde archivos o URLs, útil para visualización y análisis.


In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()#cambio de la ubicacion de busqueda de el kaggle.json para buscarlo en la raiz del proyecto
from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
import numpy as np
import shutil 
import cv2 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
from skimage import io


In [ ]:
directorio = "zargazo_dataset"

# Verifica si existe la carpeta para el data set del zargazo, si no la crea
if not os.path.exists(directorio):
    os.makedirs(directorio)
    print(f"✅ Carpeta creada: {directorio}")
else:
    print(f"ℹ️ La carpeta ya existe: {directorio}")

In [ ]:

if os.path.exists(directorio) and len(os.listdir(directorio)) > 0:
    print(f"✅ Dataset ya descargado en ./{directorio}.")
else:
    print("📥 Dataset no encontrado, descargando...")
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("irvingvasquez/publicsargazods", path=directorio, unzip=True)
    print(f"✅ Dataset descargado en ./{directorio}")

In [ ]:
ruta_labels = os.path.join(directorio, "labels", "labels.csv")
df_train = pd.read_csv(ruta_labels)
datos = df_train.iloc[:,:] # Seleccionar un subconjunto
bd_lenght, bd_width = datos.shape
datos.sample(5)

In [ ]:
# imprimir cuales son las etiquetas únicas
print(datos.label.unique())

In [ ]:
datos.info()
print('Tipo de datos de las etiquetas: ', type(datos['label'][0]))

In [ ]:
datos.describe()

In [ ]:
#graficos de distriibucion de nivel de sargazo

plt.figure(figsize=(8,4))
sns.countplot(x='label',data=datos)

In [ ]:
#graficos de distribucion de las etiquetas de las fotos

plt.figure(figsize=(8,4))
sns.countplot(x='scene',data=datos)

In [ ]:
# Imágenes de ejemplo

import random

idx = random.randint(1, bd_lenght)
imagen_ejemplo=io.imread(directorio +"/images/"+ str(datos.image_name.iloc[idx]))

plt.title("Sample image, class=" + str(datos.label.iloc[idx]))
plt.imshow(imagen_ejemplo,vmin=0,vmax=1)

In [ ]:
# Para obtener información de la imagen podemos utilizar dtype y shape 
print('La imagen es de tipo:', imagen_ejemplo.dtype)
print("Dimensiones de la imagen (high, width, channels):", imagen_ejemplo.shape)